In [1]:
import seaborn as sns

import pandas as pd
import numpy as np
from scipy.stats import skew
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
import xgboost as xgb

import warnings
warnings.simplefilter('ignore')

from scipy import stats
from scipy.stats import norm, skew #for some statistics

import os
print(os.listdir("../input"))

['train.csv', 'sample_submission.csv', 'test.csv']


# Data Acquisition
Load the data

In [2]:
%%time
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

print(train.shape, test.shape)

(200000, 202) (200000, 201)
CPU times: user 15.8 s, sys: 1.22 s, total: 17 s
Wall time: 17.1 s


# Data Preparation


In [3]:
train_id = train['ID_code']
y_train = train['target']
X_train = train.drop(['ID_code', 'target'], axis=1, inplace = False)

test_id = test['ID_code']
X_test = test.drop('ID_code', axis=1, inplace = False)

# Modelling & Training

In [4]:
# df_train = xgb.DMatrix(X_train, y_train)
# df_test = xgb.DMatrix(X_test)
# params = {"max_depth":4, "eta":0.3, 'objective': 'binary:logistic'}
# model = xgb.cv(params, df_train,  num_boost_round=10, early_stopping_rounds=5)
model_xgb = xgb.XGBRegressor(n_estimators=5, max_depth=4, learning_rate=0.5) 
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.5, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=5, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

# Evaluation
#### Using Root Mean Squared Logarithmic Error (RMSLE) evaluation function

In [6]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

train_pred = model_xgb.predict(X_train)
print('RMSLE : {:.4f}'.format(rmsle(y_train, train_pred)))

RMSLE : 0.2887


# Prediction 

In [5]:
xgb_preds = model_xgb.predict(X_test)
solution = pd.DataFrame({"ID_code":test_id, "target":xgb_preds})
solution.to_csv("santander.csv", index = False)
solution.head()

,ID_code,target
0,test_0,0.082138
1,test_1,0.089271
2,test_2,0.206476
3,test_3,0.090650
4,test_4,0.151599
